In [1]:
# Quick test: Check if we can query properties from both regions
print("\n" + "="*80)
print("TESTING PROPERTY MATCHING WITH BOTH FILES")
print("="*80)

# Test coordinates
test_properties = {
    'San Jose Property': {'lat': 37.3382, 'lon': -121.8863, 'expected': 'North'},
    'Los Angeles Property': {'lat': 34.0522, 'lon': -118.2437, 'expected': 'South'},
    'San Francisco Property': {'lat': 37.7749, 'lon': -122.4194, 'expected': 'North'},
    'San Diego Property': {'lat': 32.7157, 'lon': -117.1611, 'expected': 'South'},
    'Fresno Property': {'lat': 36.7468, 'lon': -119.7726, 'expected': 'North or South'}
}

from shapely.geometry import Point

for prop_name, prop_data in test_properties.items():
    point = Point(prop_data['lon'], prop_data['lat'])
    
    # Check which file covers this point
    in_north = (north_bounds[0] <= prop_data['lon'] <= north_bounds[2] and 
               north_bounds[1] <= prop_data['lat'] <= north_bounds[3])
    in_south = (south_bounds[0] <= prop_data['lon'] <= south_bounds[2] and 
               south_bounds[1] <= prop_data['lat'] <= south_bounds[3])
    
    print(f"\n{prop_name} ({prop_data['lat']:.4f}°, {prop_data['lon']:.4f}°):")
    print(f"  Expected: {prop_data['expected']}")
    
    if in_north and in_south:
        print(f"  ✅ Covered by BOTH files (overlap area)")
    elif in_north:
        print(f"  ✅ Covered by NORTH file")
    elif in_south:
        print(f"  ✅ Covered by SOUTH file")
    else:
        print(f"  ❌ NOT covered by either file")

print("\n" + "="*80)
print("FINAL RECOMMENDATIONS")
print("="*80)
print("\n1. ✅ Both files are in the same coordinate system (EPSG:3857)")
print("2. ✅ Files have identical column structure")
print("3. ✅ Together they cover entire California")
print("\n4. For spatial queries:")
print("   - Properties north of ~36° latitude: Use NORTH file")
print("   - Properties south of ~37° latitude: Use SOUTH file")
print("   - Properties between 36-37° latitude: Check both files")
print("\n5. For complete analysis, you should:")
print("   a) Load both files")
print("   b) Convert both to EPSG:4326")
print("   c) Query the appropriate file based on property latitude")
print("   OR")
print("   d) Combine both files into one (may be memory intensive)")


TESTING PROPERTY MATCHING WITH BOTH FILES


NameError: name 'north_bounds' is not defined

In [ ]:
# Visualize the complete California coverage with both files
fig, ax = plt.subplots(figsize=(12, 10))

# California outline (approximate)
ca_west = -124.5
ca_east = -114.0
ca_south = 32.5
ca_north = 42.0

# Plot California boundary
ax.add_patch(plt.Rectangle((ca_west, ca_south), ca_east-ca_west, ca_north-ca_south,
                          fill=False, edgecolor='black', linewidth=2, label='California Boundary'))

# Plot North file coverage
ax.add_patch(plt.Rectangle((north_bounds[0], north_bounds[1]), 
                          north_bounds[2]-north_bounds[0], 
                          north_bounds[3]-north_bounds[1],
                          alpha=0.3, facecolor='green', 
                          edgecolor='darkgreen', linewidth=2,
                          label='North File Coverage'))

# Plot South file coverage  
ax.add_patch(plt.Rectangle((south_bounds[0], south_bounds[1]), 
                          south_bounds[2]-south_bounds[0], 
                          south_bounds[3]-south_bounds[1],
                          alpha=0.3, facecolor='blue', 
                          edgecolor='darkblue', linewidth=2,
                          label='South File Coverage'))

# Add city markers
for city, (lat, lon) in cities.items():
    ax.plot(lon, lat, 'ro', markersize=8)
    ax.text(lon+0.2, lat, city, fontsize=10, ha='left')

# Check for overlap area
overlap_west = max(north_bounds[0], south_bounds[0])
overlap_east = min(north_bounds[2], south_bounds[2])
overlap_south = max(north_bounds[1], south_bounds[1])
overlap_north = min(north_bounds[3], south_bounds[3])

if overlap_south < overlap_north:  # There is overlap
    ax.add_patch(plt.Rectangle((overlap_west, overlap_south), 
                              overlap_east-overlap_west, 
                              overlap_north-overlap_south,
                              alpha=0.5, facecolor='yellow', 
                              edgecolor='orange', linewidth=2,
                              label='Overlap Area'))

ax.set_xlim(ca_west-1, ca_east+1)
ax.set_ylim(ca_south-1, ca_north+1)
ax.set_xlabel('Longitude (degrees)', fontsize=12)
ax.set_ylabel('Latitude (degrees)', fontsize=12)
ax.set_title('California Zoning Data Coverage: North and South Files', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("COVERAGE SUMMARY")
print("="*80)

# Calculate total coverage
total_west = min(north_bounds[0], south_bounds[0])
total_east = max(north_bounds[2], south_bounds[2])
total_south = min(north_bounds[1], south_bounds[1])
total_north = max(north_bounds[3], south_bounds[3])

print(f"Combined coverage:")
print(f"  West:  {total_west:.4f}°")
print(f"  South: {total_south:.4f}°")
print(f"  East:  {total_east:.4f}°")
print(f"  North: {total_north:.4f}°")

print(f"\nCalifornia bounds:")
print(f"  West:  {ca_west:.1f}°")
print(f"  South: {ca_south:.1f}°")
print(f"  East:  {ca_east:.1f}°")
print(f"  North: {ca_north:.1f}°")

# Check completeness
if (total_south <= ca_south + 0.5 and total_north >= ca_north - 0.5 and
    total_west <= ca_west + 0.5 and total_east >= ca_east - 0.5):
    print("\n✅ COMPLETE COVERAGE: Both files together cover entire California!")
else:
    print("\n⚠️  INCOMPLETE COVERAGE: Some areas of California may not be covered")
    if total_south > ca_south + 0.5:
        print(f"   Missing: Southern areas below latitude {total_south:.2f}°")
    if total_north < ca_north - 0.5:
        print(f"   Missing: Northern areas above latitude {total_north:.2f}°")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ANALYZING NORTH AND SOUTH CALIFORNIA ZONING FILES")
print("="*80)

# File paths
north_file = "../data/raw/zoning-data/ca-north/California_Statewide_Zoning_North_4449911999654225179.gpkg"
south_file = "../data/raw/zoning-data/ca-south/California_Statewide_Zoning_South_685864265303796864.gpkg"

# Load samples from both files
print("\nLoading North California file (sample)...")
gdf_north_sample = gpd.read_file(north_file, rows=1000)
print(f"✓ North file loaded: {len(gdf_north_sample)} sample records")

print("\nLoading South California file (sample)...")
gdf_south_sample = gpd.read_file(south_file, rows=1000)
print(f"✓ South file loaded: {len(gdf_south_sample)} sample records")

print("\n" + "="*60)
print("COORDINATE SYSTEM VERIFICATION")
print("="*60)

# Check CRS
print(f"North file CRS: {gdf_north_sample.crs}")
print(f"South file CRS: {gdf_south_sample.crs}")

if gdf_north_sample.crs == gdf_south_sample.crs:
    print("✅ Both files use the SAME coordinate system!")
else:
    print("⚠️  WARNING: Files use DIFFERENT coordinate systems!")
    print("   This needs to be fixed before combining data")

# Convert both to WGS84 for analysis
print("\nConverting to WGS84 for analysis...")
gdf_north_wgs84 = gdf_north_sample.to_crs('EPSG:4326')
gdf_south_wgs84 = gdf_south_sample.to_crs('EPSG:4326')

# Get bounds
north_bounds = gdf_north_wgs84.total_bounds
south_bounds = gdf_south_wgs84.total_bounds

print("\n" + "="*60)
print("GEOGRAPHIC COVERAGE ANALYSIS")
print("="*60)

print("\nNORTH FILE coverage:")
print(f"  West:  {north_bounds[0]:.4f}°")
print(f"  South: {north_bounds[1]:.4f}°")
print(f"  East:  {north_bounds[2]:.4f}°")
print(f"  North: {north_bounds[3]:.4f}°")

print("\nSOUTH FILE coverage:")
print(f"  West:  {south_bounds[0]:.4f}°")
print(f"  South: {south_bounds[1]:.4f}°")
print(f"  East:  {south_bounds[2]:.4f}°")
print(f"  North: {south_bounds[3]:.4f}°")

# Check for overlap or gap
overlap_or_gap = north_bounds[1] - south_bounds[3]
if overlap_or_gap > 0:
    print(f"\n⚠️  GAP DETECTED: {overlap_or_gap:.4f}° gap between files")
elif overlap_or_gap < 0:
    print(f"\n✅ Files OVERLAP by {abs(overlap_or_gap):.4f}° (good for complete coverage)")
else:
    print("\n✅ Files meet perfectly at boundary")

# Check key cities coverage
print("\n" + "="*60)
print("CITY COVERAGE CHECK")
print("="*60)

cities = {
    'San Francisco': (37.7749, -122.4194),
    'San Jose': (37.3382, -121.8863),
    'Los Angeles': (34.0522, -118.2437),
    'San Diego': (32.7157, -117.1611),
    'Sacramento': (38.5816, -121.4944),
    'Fresno': (36.7468, -119.7726),
    'Bakersfield': (35.3733, -119.0187)
}

for city, (lat, lon) in cities.items():
    in_north = (north_bounds[0] <= lon <= north_bounds[2]) and (north_bounds[1] <= lat <= north_bounds[3])
    in_south = (south_bounds[0] <= lon <= south_bounds[2]) and (south_bounds[1] <= lat <= south_bounds[3])
    
    if in_north:
        print(f"  {city}: ✅ Covered by NORTH file")
    elif in_south:
        print(f"  {city}: ✅ Covered by SOUTH file")
    else:
        print(f"  {city}: ❌ NOT COVERED by either file!")

print("\n" + "="*60)
print("DATA SCHEMA COMPARISON")
print("="*60)

# Compare columns
north_cols = set(gdf_north_sample.columns)
south_cols = set(gdf_south_sample.columns)

if north_cols == south_cols:
    print("✅ Both files have IDENTICAL column structure")
    print(f"   Columns: {', '.join(sorted(north_cols - {'geometry'}))}")
else:
    print("⚠️  Files have DIFFERENT columns:")
    print(f"   In North only: {north_cols - south_cols}")
    print(f"   In South only: {south_cols - north_cols}")
    print(f"   Common columns: {north_cols & south_cols}")

## Step 9: Verify North and South California Zoning Files

Now checking both North and South files in separate directories:
- North: `ca-north/California_Statewide_Zoning_North_4449911999654225179.gpkg`
- South: `ca-south/California_Statewide_Zoning_South_685864265303796864.gpkg`

In [ ]:
# Visualize the coverage problem
import matplotlib.pyplot as plt
import numpy as np

# California key cities and their latitudes
cities = {
    'Oregon Border': 42.00,
    'Eureka': 40.80,
    'Redding': 40.59,
    'Sacramento': 38.58,
    'San Francisco': 37.77,
    'San Jose': 37.34,
    'Fresno': 36.74,
    'Bakersfield': 35.37,
    'Los Angeles': 34.05,
    'San Diego': 32.72,
    'Mexican Border': 32.53
}

# GPKG coverage boundary (from our analysis)
gpkg_south_boundary = 37.48  # Southern boundary of the North file

# Create visualization
fig, ax = plt.subplots(figsize=(10, 8))

# Plot California latitude range
ax.barh(0, 42.00 - 32.53, left=32.53, height=0.5, 
        color='lightblue', label='Full California')

# Plot GPKG coverage
ax.barh(0.6, 42.00 - gpkg_south_boundary, left=gpkg_south_boundary, height=0.5,
        color='green', alpha=0.7, label='GPKG Coverage (North file)')

# Plot uncovered area
ax.barh(0.6, gpkg_south_boundary - 32.53, left=32.53, height=0.5,
        color='red', alpha=0.7, label='NOT Covered')

# Add city markers
for city, lat in cities.items():
    ax.axvline(x=lat, color='gray', linestyle='--', alpha=0.5)
    ax.text(lat, 1.3, city, rotation=45, ha='left', va='bottom', fontsize=9)
    
    # Color code the text
    if lat >= gpkg_south_boundary:
        color = 'green'
    else:
        color = 'red'
    ax.plot(lat, 1.2, 'o', color=color, markersize=8)

# Highlight the problem
ax.axvspan(32.53, gpkg_south_boundary, alpha=0.2, color='red')
ax.text(34.5, 0.3, 'NOT COVERED\n(includes LA)', 
        fontsize=12, fontweight='bold', color='red', ha='center')

ax.text(39, 0.3, 'COVERED\n(North file)', 
        fontsize=12, fontweight='bold', color='green', ha='center')

# Labels and formatting
ax.set_xlim(32, 43)
ax.set_ylim(-0.2, 1.5)
ax.set_xlabel('Latitude (degrees)', fontsize=12)
ax.set_title('GPKG Zoning File Coverage Problem', fontsize=14, fontweight='bold')
ax.set_yticks([])
ax.legend(loc='upper right')
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("="*70)
print("COVERAGE ANALYSIS RESULT")
print("="*70)
print(f"✅ Northern California cities (covered):")
for city, lat in cities.items():
    if lat >= gpkg_south_boundary:
        print(f"   - {city}: {lat:.2f}°")
        
print(f"\n❌ Southern/Central California cities (NOT covered):")
for city, lat in cities.items():
    if lat < gpkg_south_boundary:
        print(f"   - {city}: {lat:.2f}°")
        
print("\n" + "="*70)
print("SOLUTION NEEDED")
print("="*70)
print("You need to find and download:")
print("  California_Statewide_Zoning_SOUTH_*.gpkg")
print("  to get zoning data for Los Angeles and Southern California")

## Step 8: GPKG Coverage Analysis - Northern California Only!

⚠️ **IMPORTANT DISCOVERY**: The zoning GPKG file only covers **Northern California**!

- Filename: `California_Statewide_Zoning_North_4449911999654225179.gpkg`
- Note the word **"North"** in the filename
- Coverage: Approximately north of latitude 37.48° (near San Francisco Bay Area)
- **Los Angeles County is NOT covered** (LA is at 34.05° latitude)

# Coordinate System Validation for county_dfs

This notebook verifies that the longitude/latitude coordinates in county_dfs are indeed in EPSG:4326 (WGS84) format.

In [ ]:
import json
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from IPython.display import display

# Set display options
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)

## Step 1: Load Data (same as agentic-demo.ipynb)

In [ ]:
# Load the data
data_dir = "../data/raw/"
county_files = {
    "Santa Clara": "20250822_114114_santa_clara_county_ca_santa_clara_county.json",
    "Alameda": "20250822_114052_alameda_county_ca_alameda_county.json", 
    "Los Angeles": "20250822_113940_los_angeles_county_ca_los_angeles_county.json"
}

# Load and process data
county_data = {}
for county_name, filename in county_files.items():
    filepath = os.path.join(data_dir, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        county_data[county_name] = data

# Extract coordinates
all_coords = []
for county_name, data in county_data.items():
    for listing in data['listings'][:100]:  # Sample first 100 from each county
        coord = listing.get('location', {}).get('address', {}).get('coordinate', {})
        if coord:
            all_coords.append({
                'county': county_name,
                'longitude': coord.get('lon', 0),
                'latitude': coord.get('lat', 0),
                'address': listing.get('location', {}).get('address', {}).get('line', ''),
                'city': listing.get('location', {}).get('address', {}).get('city', '')
            })

coords_df = pd.DataFrame(all_coords)
coords_df = coords_df[(coords_df['longitude'] != 0) & (coords_df['latitude'] != 0)]
print(f"Loaded {len(coords_df)} valid coordinates")

Loaded 300 valid coordinates


## Step 2: Verify Coordinate Ranges

WGS84 (EPSG:4326) characteristics:
- Longitude: -180 to +180 degrees
- Latitude: -90 to +90 degrees
- California approximate bounds: 
  - Longitude: -124.5 to -114
  - Latitude: 32.5 to 42

In [ ]:
# Check coordinate ranges
print("=" * 60)
print("COORDINATE RANGE ANALYSIS")
print("=" * 60)

print(f"\nLongitude range: {coords_df['longitude'].min():.6f} to {coords_df['longitude'].max():.6f}")
print(f"Latitude range: {coords_df['latitude'].min():.6f} to {coords_df['latitude'].max():.6f}")

# Check if values are within WGS84 global bounds
wgs84_valid = (
    (coords_df['longitude'] >= -180) & (coords_df['longitude'] <= 180) &
    (coords_df['latitude'] >= -90) & (coords_df['latitude'] <= 90)
).all()

print(f"\n✓ All coordinates within WGS84 bounds (-180 to 180, -90 to 90): {wgs84_valid}")

# Check if values are within California bounds
ca_bounds = {
    'lon_min': -124.5, 'lon_max': -114.0,
    'lat_min': 32.5, 'lat_max': 42.0
}

ca_valid = (
    (coords_df['longitude'] >= ca_bounds['lon_min']) & 
    (coords_df['longitude'] <= ca_bounds['lon_max']) &
    (coords_df['latitude'] >= ca_bounds['lat_min']) & 
    (coords_df['latitude'] <= ca_bounds['lat_max'])
).all()

print(f"✓ All coordinates within California bounds: {ca_valid}")

# If these were Web Mercator (EPSG:3857), values would be in millions
print("\n" + "="*60)
print("COMPARISON WITH OTHER COORDINATE SYSTEMS")
print("="*60)
print("\nIf these were EPSG:3857 (Web Mercator):")
print("  Expected range: -20,000,000 to 20,000,000 (meters)")
print("  Our range: CLEARLY NOT in millions, so NOT EPSG:3857")

print("\nIf these were State Plane (e.g., EPSG:2227):")
print("  Expected range: millions of feet")
print("  Our range: Small decimal values, so NOT State Plane")

COORDINATE RANGE ANALYSIS

Longitude range: -122.289657 to -117.722653
Latitude range: 33.676520 to 37.897645

✓ All coordinates within WGS84 bounds (-180 to 180, -90 to 90): True
✓ All coordinates within California bounds: True

COMPARISON WITH OTHER COORDINATE SYSTEMS

If these were EPSG:3857 (Web Mercator):
  Expected range: -20,000,000 to 20,000,000 (meters)
  Our range: CLEARLY NOT in millions, so NOT EPSG:3857

If these were State Plane (e.g., EPSG:2227):
  Expected range: millions of feet
  Our range: Small decimal values, so NOT State Plane


## Step 3: Sample Point Verification

In [ ]:
# Take specific known locations and verify
print("="*60)
print("SAMPLE POINT VERIFICATION")
print("="*60)

# Sample a few points
samples = coords_df.head(5)

for idx, row in samples.iterrows():
    print(f"\nAddress: {row['address']}, {row['city']}")
    print(f"  Coordinates: ({row['longitude']:.6f}, {row['latitude']:.6f})")
    
    # These should be negative longitude (west) and positive latitude (north) for California
    if row['longitude'] < 0:
        print("  ✓ Longitude is negative (West of Prime Meridian) - CORRECT for California")
    else:
        print("  ✗ Longitude is positive - WRONG for California")
        
    if 32 < row['latitude'] < 42:
        print("  ✓ Latitude is between 32-42 degrees - CORRECT for California")
    else:
        print("  ✗ Latitude outside California range") 

SAMPLE POINT VERIFICATION

Address: 5939 Pala Mesa Dr, San Jose
  Coordinates: (-121.813919, 37.240225)
  ✓ Longitude is negative (West of Prime Meridian) - CORRECT for California
  ✓ Latitude is between 32-42 degrees - CORRECT for California

Address: 119 Belcrest Dr, Los Gatos
  Coordinates: (-121.914295, 37.230329)
  ✓ Longitude is negative (West of Prime Meridian) - CORRECT for California
  ✓ Latitude is between 32-42 degrees - CORRECT for California

Address: 2055 Rucker Ave, Gilroy
  Coordinates: (-121.556420, 37.062856)
  ✓ Longitude is negative (West of Prime Meridian) - CORRECT for California
  ✓ Latitude is between 32-42 degrees - CORRECT for California

Address: 5850 Chesbro Ave, San Jose
  Coordinates: (-121.843059, 37.244173)
  ✓ Longitude is negative (West of Prime Meridian) - CORRECT for California
  ✓ Latitude is between 32-42 degrees - CORRECT for California

Address: 1641 Rue Avati, San Jose
  Coordinates: (-121.880692, 37.388961)
  ✓ Longitude is negative (West of Pr

## Step 4: Create GeoDataFrame and Verify CRS

In [ ]:
# Create GeoDataFrame with explicit EPSG:4326
geometry = [Point(xy) for xy in zip(coords_df.longitude, coords_df.latitude)]
gdf_4326 = gpd.GeoDataFrame(coords_df, crs='EPSG:4326', geometry=geometry)

print("="*60)
print("GEODATAFRAME CRS VERIFICATION")
print("="*60)
print(f"\nGeoDataFrame CRS: {gdf_4326.crs}")
print(f"CRS Name: {gdf_4326.crs.name if gdf_4326.crs else 'None'}")

# Get bounds
bounds = gdf_4326.total_bounds
print(f"\nTotal bounds:")
print(f"  Min X (West): {bounds[0]:.6f}°")
print(f"  Min Y (South): {bounds[1]:.6f}°")
print(f"  Max X (East): {bounds[2]:.6f}°")
print(f"  Max Y (North): {bounds[3]:.6f}°")

GEODATAFRAME CRS VERIFICATION

GeoDataFrame CRS: EPSG:4326
CRS Name: WGS 84

Total bounds:
  Min X (West): -122.289657°
  Min Y (South): 33.676520°
  Max X (East): -117.722653°
  Max Y (North): 37.897645°


## Step 5: Transform to Other CRS to Demonstrate Difference

In [ ]:
# Transform to EPSG:3857 (Web Mercator) to show the difference
gdf_3857 = gdf_4326.to_crs('EPSG:3857')

print("="*60)
print("COORDINATE TRANSFORMATION COMPARISON")
print("="*60)

# Compare first point in both systems
first_point_4326 = gdf_4326.iloc[0]
first_point_3857 = gdf_3857.iloc[0]

print("\nFirst point comparison:")
print(f"Address: {first_point_4326['address']}, {first_point_4326['city']}")
print(f"\nEPSG:4326 (WGS84 - Degrees):")
print(f"  X (Longitude): {first_point_4326.geometry.x:.6f}°")
print(f"  Y (Latitude): {first_point_4326.geometry.y:.6f}°")

print(f"\nEPSG:3857 (Web Mercator - Meters):")
print(f"  X: {first_point_3857.geometry.x:.2f} meters")
print(f"  Y: {first_point_3857.geometry.y:.2f} meters")

print("\n" + "="*60)
print("CONCLUSION")
print("="*60)
print("\n✓ The coordinates in county_dfs are DEFINITELY EPSG:4326 (WGS84) because:")
print("  1. Values are in decimal degrees (not millions of meters)")
print("  2. Longitude values are negative (West of Prime Meridian)")
print("  3. Values fall within California's geographic bounds")
print("  4. Transformation to EPSG:3857 produces vastly different values")

COORDINATE TRANSFORMATION COMPARISON

First point comparison:
Address: 5939 Pala Mesa Dr, San Jose

EPSG:4326 (WGS84 - Degrees):
  X (Longitude): -121.813919°
  Y (Latitude): 37.240225°

EPSG:3857 (Web Mercator - Meters):
  X: -13560263.43 meters
  Y: 4472644.16 meters

CONCLUSION

✓ The coordinates in county_dfs are DEFINITELY EPSG:4326 (WGS84) because:
  1. Values are in decimal degrees (not millions of meters)
  2. Longitude values are negative (West of Prime Meridian)
  3. Values fall within California's geographic bounds
  4. Transformation to EPSG:3857 produces vastly different values


## Step 6: Visual Verification with Interactive Map

In [ ]:
# Create an interactive map to visually verify locations
# Center map on California
ca_center = [36.7783, -119.4179]
m = folium.Map(location=ca_center, zoom_start=6)

# Add sample points to map
sample_points = coords_df.sample(min(20, len(coords_df)))

for idx, row in sample_points.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=f"{row['address']}, {row['city']}",
        tooltip=f"{row['city']} ({row['county']} County)"
    ).add_to(m)

print("Interactive map with sample points:")
print("If points appear in correct California cities, coordinates are confirmed as WGS84")
display(m)

Interactive map with sample points:
If points appear in correct California cities, coordinates are confirmed as WGS84


## Step 7: Final Verification - Known Landmarks

In [ ]:
# Compare with known California landmarks in WGS84
known_landmarks = {
    'San Francisco City Hall': (-122.4194, 37.7793),
    'Los Angeles City Hall': (-118.2437, 34.0537),
    'San Jose City Hall': (-121.8863, 37.3382),
    'Oakland City Hall': (-122.2728, 37.8053)
}

print("="*60)
print("VERIFICATION AGAINST KNOWN LANDMARKS")
print("="*60)

for landmark, (lon, lat) in known_landmarks.items():
    print(f"\n{landmark}:")
    print(f"  Known WGS84: ({lon:.4f}, {lat:.4f})")
    
    # Find nearest point in our data
    city = landmark.split(' City')[0]
    city_data = coords_df[coords_df['city'].str.contains(city, case=False, na=False)]
    
    if not city_data.empty:
        avg_lon = city_data['longitude'].mean()
        avg_lat = city_data['latitude'].mean()
        print(f"  Our data avg for {city}: ({avg_lon:.4f}, {avg_lat:.4f})")
        
        # Calculate difference
        diff_lon = abs(avg_lon - lon)
        diff_lat = abs(avg_lat - lat)
        
        if diff_lon < 0.5 and diff_lat < 0.5:  # Within ~50km
            print(f"  ✓ Close match - confirms WGS84")
        else:
            print(f"  Distance: {diff_lon:.4f}° lon, {diff_lat:.4f}° lat")

VERIFICATION AGAINST KNOWN LANDMARKS

San Francisco City Hall:
  Known WGS84: (-122.4194, 37.7793)

Los Angeles City Hall:
  Known WGS84: (-118.2437, 34.0537)
  Our data avg for Los Angeles: (-118.2953, 34.0460)
  ✓ Close match - confirms WGS84

San Jose City Hall:
  Known WGS84: (-121.8863, 37.3382)
  Our data avg for San Jose: (-121.8583, 37.2893)
  ✓ Close match - confirms WGS84

Oakland City Hall:
  Known WGS84: (-122.2728, 37.8053)
  Our data avg for Oakland: (-122.2034, 37.8013)
  ✓ Close match - confirms WGS84
